# Download a random pro's CFG and place it in CSGO folder
## Set CSGO Path

In [18]:
# Change to match your CSGO install directory
PATH = "H:\SteamLibrary\steamapps\common\Counter-Strike Global Offensive"

## Download/Create Dataframe

In [19]:
from download_table import get_table


dataframe = get_table()


# Filter out rows where "Team" column isn't one of:

team_list = [
    "Team Spirit",
    "Team Vitality",
    "Natus Vincere",
    "Eternal Fire",
    "G2 Esports",
    "FaZe Clan",
    "The Mongolz",
    "MOUZ",
    "Astralis",
    "Falcons Esports",
]

missing_teams = [team for team in team_list if team not in dataframe["Team"].unique()]

if missing_teams:

    print("Missing teams:", missing_teams)

    raise ValueError("Missing teams")

dataframe = dataframe[dataframe["Team"].isin(team_list)]

## Download CFGS

In [ ]:
import multiprocessing
import os
import requests
import threading
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Extract data
def get_player_data(row):
    # Go to the player's profile page
    player = row["Player"]
    team = row["Team"]
    player_url = f"https://prosettings.net/players/{player}"
    
    # Setup Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize WebDriver
    driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"), options=chrome_options)
    
    settings_dict = {}
    
    try:
        print(f"Fetching data for {player}...")
        driver.get(player_url)
        
        # Wait for page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "section"))
        )
        
        # Get page source after JS execution
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Process each section
        section_ids = ["cs2_mouse", "cs2_crosshair", "cs2_viewmodel", "cs2_bob", 
                      "cs2_video_settings", "advanced_video"]
        
        for section_id in section_ids:
            section = soup.find("section", id=section_id)
            if section:
                table = section.find("table", class_="settings")
                if table:
                    for row in table.find_all("tr"):
                        if row.has_attr("data-field"):
                            key = row["data-field"]
                            value_cell = row.find("td")
                            if value_cell:
                                value = value_cell.text.strip()
                                settings_dict[key] = value
        
        # Create pro_configs/{team} if it doesn't exist
        os.makedirs(f"pro_configs/{team}", exist_ok=True)
        
        # Save to cfg file
        with open(f"pro_configs/{team}/{player}.cfg", "w") as f:
            f.write(f"// {player} CFG\n")
            for key, value in settings_dict.items():
                f.write(f"{key} {value}\n")
        
        with open(f"pro_configs/{team}/{player}.cfg", "a") as f:
            f.write("\n// Print settings\nclear\n")
            for key, value in settings_dict.items():
                f.write(f"echo {key}: {value}\n")
        
        print(f"Successfully saved config for {player}")
    
    except Exception as e:
        print(f"Error processing {player}: {str(e)}")
    
    finally:
        driver.quit()

# Test with one player
# print(f"Starting with {dataframe.iloc[0]['Player']}")
# get_player_data(dataframe.iloc[0])
# print(f"Done with {dataframe.iloc[0]['Player']}")

# Run with all players
players = dataframe
threads = []

max_threads = multiprocessing.cpu_count()

for index, row in players.iterrows():
    while threading.active_count() > max_threads:
        time.sleep(1)  # Wait for an available thread slot
    thread = threading.Thread(target=get_player_data, args=(row,))
    threads.append(thread)
    thread.start()
    time.sleep(1)  # Add delay to avoid overwhelming the server

for thread in threads:
    thread.join()

C:\Users\Walks\AppData\Local\Temp\ipykernel_6464\1379835480.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"), options=chrome_options)


Fetching data for HeavyGod...
Fetching data for huNter...
Fetching data for m0NESY...
Fetching data for Snax...
Fetching data for malbsMd...
Fetching data for iM...
Fetching data for Aleksib...
Fetching data for b1t...
Fetching data for jL...
Fetching data for s1mple...
Fetching data for w0nderful...
Fetching data for magixx...
Fetching data for zont1x...
Successfully saved config for m0NESY
Fetching data for sh1ro...
Fetching data for chopper...
Successfully saved config for HeavyGod
Fetching data for donk...
Fetching data for Brollan...
Fetching data for xertioN...
Fetching data for Jimpphat...
Fetching data for Spinx...
Successfully saved config for huNter
Fetching data for torzsi...
Fetching data for broky...
Fetching data for Frozen...
Fetching data for karrigan...
Fetching data for olofmeister...
Fetching data for rain...
Fetching data for EliGE...
Fetching data for ropz...
Fetching data for apEX...
Fetching data for mezii...
Fetching data for flameZ...
Successfully saved config 

Exception in thread Thread-196 (get_player_data):
Traceback (most recent call last):
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Walks\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Walks\AppData\Local\Temp\ipykernel_6464\1379835480.py", line 30, in get_player_data
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 70, in __init__
    super(WebDriver, self).__init__(DesiredCapabilities.CHROME['browserName'], "goog",
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 92, in __init__
    RemoteWebDr

Successfully saved config for xertioN


Exception in thread Thread-195 (get_player_data):
Traceback (most recent call last):
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Walks\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Walks\AppData\Local\Temp\ipykernel_6464\1379835480.py", line 30, in get_player_data
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 70, in __init__
    super(WebDriver, self).__init__(DesiredCapabilities.CHROME['browserName'], "goog",
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 92, in __init__
    RemoteWebDr

...

Exception in thread Thread-202 (get_player_data):
Traceback (most recent call last):
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Walks\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Walks\AppData\Local\Temp\ipykernel_6464\1379835480.py", line 30, in get_player_data
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 70, in __init__
    super(WebDriver, self).__init__(DesiredCapabilities.CHROME['browserName'], "goog",
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 92, in __init__
    RemoteWebDr

Successfully saved config for donk


  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 70, in __init__
    super(WebDriver, self).__init__(DesiredCapabilities.CHROME['browserName'], "goog",
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 92, in __init__
    RemoteWebDriver.__init__(
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 275, in __init__
    self.start_session(capabilities, browser_profile)
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 365, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line

Successfully saved config for apEX
Successfully saved config for Frozen
Fetching data for bLitz...
Fetching data for 910...
Fetching data for Senzu...
Fetching data for dupreeh...
Successfully saved config for broky
Fetching data for TeSeS...
Successfully saved config for sh1ro
Fetching data for NiKo...
Fetching data for kyxsan...
Successfully saved config for mezii
Successfully saved config for torzsi
Fetching data for Techno4K...
Successfully saved config for karrigan
Fetching data for degster...
Fetching data for kennyS...
Successfully saved config for chopper
Fetching data for Magisk...


Exception in thread Thread-205 (get_player_data):
Traceback (most recent call last):
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Walks\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Walks\AppData\Local\Temp\ipykernel_6464\1379835480.py", line 30, in get_player_data
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 70, in __init__
    super(WebDriver, self).__init__(DesiredCapabilities.CHROME['browserName'], "goog",
  File "c:\Users\Walks\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 92, in __init__
    RemoteWebDr

## Pick a random CFG

In [ ]:
# Pick random .cfg from pro_configs/*
import random
import os

# Get list of all .cfg files
cfgs = []
for root, dirs, files in os.walk("pro_configs"):
    for file in files:
        if file.endswith(".cfg"):
            cfgs.append(os.path.join(root, file))

# Pick random .cfg
random_cfg = random.choice(cfgs)
print(random_cfg)

pro_configs\FaZe Clan\EliGE.cfg
